In [1]:
# utils and general stuff
import pandas as pd
import numpy as np
from exam_utils import timeParser
import re
from sklearn.metrics import cohen_kappa_score

#Packages to create DFM
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import TweetTokenizer

#Models to train
from sklearn.linear_model import LogisticRegression

#Packages for cross-validation and parameter tuning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('lemma_all.csv', compression='zip')

In [3]:
df.loc[:, 'tweet_created_at'] = df.tweet_created_at.apply(timeParser)

In [4]:
# subsetting only after the electiong and making a copy to get rid of the setting with copy warning
df_ae = df.loc[df.tweet_created_at > '2019-06-05'].copy()  

In [5]:
df_ae.loc[:, 'tweet_id'] = df_ae.loc[:, 'tweet_id'].astype(int)

In [6]:
# removing all retweets
df_ae = df_ae.loc[~df_ae.tweet_full_text.str.contains('^RT')]

# dropping nans in tweet lemma
df_ae = df_ae.dropna(subset=['tweet_text_lemma'])

In [30]:
def sample_dataset(df, n=100, random_state=42):
    '''Takes in a df and returns 100 random tweets to be labelled'''
    temp = df.sample(n, random_state=random_state)
    temp.loc[:, 'label'] = np.nan
    return temp

In [31]:
# Getting the first dataset to label!
label = sample_dataset(df_ae)

In [32]:
label.to_excel('label_this.xlsx')

# Active learning loop

## importing data and splitting into X and y

In [19]:
def split_data(path, test_data=False):
    '''takes in the path to the latest labelled data set and returns X_train, y_train, and a df
    could have used train_test_split'''
    new_df = pd.read_excel(path, index_col=0)
    X = new_df.tweet_text_lemma
    y = new_df.label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    if test_data:
        return X_train, X_test, y_train, y_test, new_df
    else:
        return X, y, new_df

In [7]:
def get_unlabelled(new_df, old_df):
    '''takes in the new df and removes the ones in the new one from the old one'''
    unlabelled_df = old_df.loc[~old_df.index.isin(new_df.index)]
    return unlabelled_df

In [9]:
X_train, y_train, labelled_df = split_data('label.xlsx')

In [10]:
unlabelled_df = get_unlabelled(labelled_df, df_ae)

In [8]:
# pipeline to train on
tokenizer = TweetTokenizer()

pipeline = Pipeline([ 
    ('cv', CountVectorizer(tokenizer=tokenizer.tokenize, ngram_range = (1, 2), max_df=0.999, min_df=0.01)),
    ('tfidf', TfidfTransformer(use_idf = False)),
    ('logreg', LogisticRegression())
])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('cv',
                 CountVectorizer(max_df=0.999, min_df=0.01, ngram_range=(1, 2),
                                 tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7faac89cc310>>)),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('logreg', LogisticRegression())])

In [10]:
def predict_unlabelled(pipeline, unlabelled_df):
    '''takes in a pipeline, the unlabelled df and adds the maximum probability column
    Then it sorts the dataframe by max proba and returns it'''
    # predicts for the three classes for all entries in the dataset
    predictions = pipeline.predict_proba(unlabelled_df.tweet_text_lemma)
    # creates a column with the max probability
    temp = unlabelled_df.copy()
    temp.loc[:, 'max_proba'] = [max(pred) for pred in predictions]
    return temp

In [14]:
unlabelled_df = predict_unlabelled(pipeline, unlabelled_df)


In [17]:
def label_new_set(unlabelled_df, labelled_df, new_name):
    '''takes in the df produced above, sorts it and saves a new df to be labelled'''
    unlabelled_df.sort_values(by='max_proba', inplace=True)
    new_df = unlabelled_df[:100].copy()
    new_df.loc[:, 'label'] = np.nan
    new_df = pd.concat([new_df, labelled_df])
    new_df.to_excel(f'{new_name}.xlsx')
    return None

In [18]:
label_new_set(unlabelled_df, labelled_df, 'label7')

## Checking the current score

# NEW PLANS

Implement function that calculates [Cohens kappa](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) between the unlabelled set between iterations; Wiedemann proposes to stop when it reaches 0.99 after three revisions; we probably need to take a bit lower. If we do this we have actually implemented one full method from the course. This also gives us a way out of labelling data :) 

In [12]:
def cohens_kappa(path_to_labelled, full_df):
    '''takes in a labelled dict and an unlabelled dict. Fits a model for each hundred labelled entries
    predicts on the unlabelled set and then calculates cohens kappa for each models prediction
    and the former iterations and returns a list of cohens kappa scores for each iteration
    Could also be augmented to print the score each iteration'''
    df = pd.read_excel(path_to_labelled, index_col=0)
    unlabelled_df = get_unlabelled(df, full_df)
    X_test = unlabelled_df.tweet_text_lemma
    predictions = []
    
    n = df.shape[0]
    for i in range(100, n + 100, 100):
        # creates a temp df with only the n lowest labelled examples
        temp_df = df.tail(i).copy()
        X = temp_df.tweet_text_lemma
        y = temp_df.label
        pipeline.fit(X, y)
        predictions.append(pipeline.predict(X_test))
    kappas = []
    for i, prediction in enumerate(predictions):
        if (i + 1) == len(predictions):
            break
        else:
            kappas.append(cohen_kappa_score(prediction, predictions[i + 1]))
    return kappas

In [25]:
df_anton.shape[0]
for i in range(100, 900, 100):
    print(i)

100
200
300
400
500
600
700
800


In [13]:
kappas = cohens_kappa('label8.xlsx', df_ae)

In [14]:
kappas

[0.0,
 0.6867892222348115,
 0.6036853561752391,
 0.4810620065186614,
 0.6829383093527561,
 0.7337599102137073,
 0.7924861553236712,
 0.8159139171172582]

## Leftovers

In [ ]:
#Fill in the parameter values in the grid 
parameter_grid = {
    'tfidf__use_idf': [False, True],
    'logreg__penalty': ['l1', 'l2', 'elasticnet'],
    'logreg__C': [0.1, 0.5, 1],
}

#Initializing a kfold with 5 folds
cv = StratifiedKFold(n_splits=5)

#Initializing the GridSearchCV
search = GridSearchCV(pipeline, parameter_grid, cv=cv, n_jobs = -1, verbose=10)

In [ ]:
search.fit(X_train, y_train)

In [53]:
pipeline.score(X_train, y_train)

0.778

In [55]:
y_pred = pipeline.predict(X_train)

In [57]:
conf = confusion_matrix(y_train, y_pred)

In [58]:
print(conf)

[[209   3   0   3]
 [ 51  75   1   5]
 [ 35   6  36   1]
 [  5   0   1  69]]


## Cohens K: AE & EG

In [2]:
!ls

Active Learning.ipynb             label3.xlsx
First scrape attempts.ipynb       label4.xlsx
Hsbm keyword generator.ipynb      label5.xlsx
Initial network.xlsx              label6.csv
Preprocessing.ipynb               label6.xlsx
Supervised learning keras.ipynb   label6_emilie.xlsx
Supervised learning.ipynb         label7.xlsx
__pycache__                       label7_ae.xlsx
app_cred.py                       label7_eg.xlsx
attack                            label_this.xlsx
exam_utils.py                     lemma_all.csv
final_final_keywords.csv          models
final_keywords.csv                readme.md
graph_full_dataset_grouped.xml.gz results
green_words.txt                   test_trainer
hSBM_Topicmodel                   topics_100_000_nostopwords.csv
hsbm_df.zip                       topics_grouped_dataset.csv
keywords_qual.xlsx                tweets_final.zip
label2.xlsx                       w2v


In [5]:
df_anton = pd.read_excel('label7_ae.xlsx', index_col=0)
df_emilie = pd.read_excel('label7_eg.xlsx', index_col=0)

In [21]:
df_anton.to_csv('label7_totest.csv')

In [12]:
labels_a = df_anton.head(200).label
labels_e = df_emilie.head(200).label

In [13]:
cohen_kappa_score(labels_a, labels_e)

0.6644647163678306

In [31]:
df_anton.loc[:, 'emilies_labels'] = df_emilie.loc[:, 'label'].copy()

In [33]:
df_anton.loc[df_anton.label != df_anton.emilies_labels].to_csv('uenige_eg_an.csv')